## Linear regression

In [1]:
import logging
import os

import numpy as np
import matplotlib.pyplot as plt
# For reproducibility
# pylint: disable=wrong-import-position
np.random.seed(42)
from dotenv import find_dotenv, load_dotenv

import fault_tolerant_ml as ft
from fault_tolerant_ml.layers import Layer
from fault_tolerant_ml.losses import MSELoss
from fault_tolerant_ml.metrics import accuracy_scorev2, confusion_matrix
from fault_tolerant_ml.optimizers import SGD, Adam
from fault_tolerant_ml.data import yearmsd

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
class LinearRegressionV2(ft.ModelV2):
    """Linear regression
    """
    def __init__(
            self,
            optimizer,
            strategy=None,
            batch_size=64,
            max_iter=300,
            shuffle=True,
            verbose=10,
            **kwargs):
        super().__init__(
            optimizer=optimizer,
            strategy=strategy, 
            batch_size=batch_size,
            max_iter=max_iter, 
            shuffle=shuffle, 
            verbose=verbose, 
            **kwargs
        )

        self.add(Layer(90, 1, activation="linear"))

In [20]:
def train(model, X_train, y_train, X_test, y_test, batch_size=32, epochs=100):
    for epoch in np.arange(epochs):
        epoch_loss = 0.0
        n_batches = 0
        for start in range(0, X_train.shape[0], batch_size):
            end = start + batch_size

            X_batch = X_train[start:end]
            y_batch = y_train[start:end]

            # Feedforward
            y_pred = model.forward(X_batch)
            
            # print(y_pred[1])

            # Calculate loss
            batch_loss = loss.loss(y_batch, y_pred, reduce=True).data

            # Backprop
            optimizer.compute_gradients(model, y_batch, y_pred)

            # Update gradients
            optimizer.apply_gradients(model)

            epoch_loss = epoch_loss + batch_loss
            n_batches += 1

        epoch_loss = epoch_loss / n_batches

        # Calculate accuracy
        y_pred_train = model.forward(X_train)
        # train_acc = accuracy_scorev2(y_train.data, y_pred_train.data)
        # Test accuracy
        y_pred_test = model.forward(X_test)
        test_loss = loss.loss(y_test, y_pred_test, reduce=True).data

        if epoch % 1 == 0:
            print(f"Iteration {epoch}: Loss={epoch_loss:.4f}, test_loss={test_loss:.4f}")
        epoch += 1
        
    return epoch_loss

In [32]:
loss = MSELoss()

# Define optimizer
optimizer = None

optimizer = SGD(
    loss=loss, 
    learning_rate=0.01
)

model = LinearRegressionV2(
    optimizer=optimizer
)

In [33]:
(x_train, y_train), (x_test, y_test) = yearmsd.load_data()

In [23]:
y_train.shape

(463715, 1)

In [34]:
x_train = ft.operators.Tensor(x_train)
y_train = ft.operators.Tensor(y_train)
x_test = ft.operators.Tensor(x_test)
y_test = ft.operators.Tensor(y_test)

In [35]:
train(
    model,
    x_train,
    y_train,
    x_test,
    y_test,
    epochs=10
)

Iteration 0: Loss=7903.2359, test_loss=49.6186
Iteration 1: Loss=47.4896, test_loss=49.6186
Iteration 2: Loss=47.4896, test_loss=49.6186
Iteration 3: Loss=47.4896, test_loss=49.6186
Iteration 4: Loss=47.4896, test_loss=49.6186
Iteration 5: Loss=47.4896, test_loss=49.6186
Iteration 6: Loss=47.4896, test_loss=49.6186
Iteration 7: Loss=47.4896, test_loss=49.6186
Iteration 8: Loss=47.4896, test_loss=49.6186
Iteration 9: Loss=47.4896, test_loss=49.6186


47.48961531054774